In [10]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Dropdown, Button
from ipywidgets import Output, HBox, Label, Text, RadioButtons, HTML, FileUpload

import ipywidgets as widgets

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import os
import re

from IPython.display import clear_output
from IPython.display import display, HTML
from IPython.display import Image

import random
import base64

from sklearn.decomposition import PCA
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.graph_objects as go

from dominate import tags

import codecs
import io

from urllib.parse import urlparse
from urllib.parse import parse_qs


In [ ]:
def HTML_with_style(df, style=None, random_id=None):
    from IPython.display import HTML
    import numpy as np
    import re

    df_html = df.to_html()

    if random_id is None:
        random_id = 'id%d' % np.random.choice(np.arange(1000000))

    if style is None:
        style = """
        <style>
            table#{random_id} {{color: black}}
        </style>
        """.format(random_id=random_id)
    else:
        new_style = []
        s = re.sub(r'</?style>', '', style).strip()
        for line in s.split('\n'):
                line = line.strip()
                if not re.match(r'^table', line):
                    line = re.sub(r'^', 'table ', line)
                new_style.append(line)
        new_style = ['<style>'] + new_style + ['</style>']

        style = re.sub(r'table(#\S+)?', 'table#%s' % random_id, '\n'.join(new_style))

    df_html = re.sub(r'<table', r'<table id=%s ' % random_id, df_html)

    return HTML(style + df_html)

In [3]:
def set_password(password):
    global pwd, version
    pwd = password
    version = "v02.2_010222"
    
set_password('password')

In [4]:
def check_password():
    if( pwd == 'password'):
        return True
    else:
        raise ValueError('Password incorrect')

In [16]:
GS_exp = 'umcu_rnaseq'

url = os.environ['VOILA_REQUEST_URL']
global current_sample_ID, specie, fragment, gene_ID
current_sample_ID = parse_qs(urlparse(url).query)['sample_id'][0]
tissue = parse_qs(urlparse(url).query)['specie'][0]
fragments = parse_qs(urlparse(url).query)['fragment'][0]
gene_ID = parse_qs(urlparse(url).query)['gene_id'][0]

main_path = './outrider/'
file = 'umcu_rnaseq_' + tissue + '_res_outrider_' + fragments + '_counts.tsv.gz'
#        file = 'umcu_rnaseq_' + tissue + '_res_outrider_' + fragments + '_ref' + current_sample_ID +'.tsv.gz'
path = main_path + file
filename = path

stats = pd.read_csv(path, compression='gzip')

# print(path)
# print(filename)

Url()

umcu_rnaseq_metadata.csv


<Response [200]>

Gene of interest plot

In [ ]:
def gene_of_interest_plot(search_gene):
    check_password()
    
    #stats = stats.loc[stats['sampleID'] == current_sample_ID]
    df = stats
    
    if search_gene.strip(' ') == '':
        print("Submit gene name.")
        
    else:      
        df = df.loc[stats['sampleID'] == current_sample_ID]
        # define chr.start position
        df = df.assign(start1 = df['geneID'].apply(lambda x: x.split('_')[1]  ))        
#        df = df.assign(start1 = df['geneID'].apply(lambda x: x.split('.')[1]  ))        
        #df = df.assign(exon_id = df['geneENSG'])        
        df = df.loc[ df['gene'] == search_gene]

        df_gene = pd.read_csv(path)
        df_gene = df_gene.loc[ df_gene['gene'] == search_gene]
        df_gene = df_gene.assign(start1 = df_gene['geneID'].apply(lambda x: x.split('_')[1]  ))        
        df_gene = df_gene.assign(exon_id = df_gene['geneENSG'])        
        df_gene.sort_values('start', axis = 0, inplace=True, ascending =True)
        df_sample = df_gene.loc[ df_gene['sampleID'] == current_sample_ID]      

        if( df.empty):
            print("Invalid gene name or gene not in data")
            
        else:
            df.sort_values('geneID', axis = 0, inplace=True, ascending =True)

            # plot  
            if re.search('intron',filename):
                x = df['geneID']
                y = df['zScore']
               
            else:
                x = df_sample['start1'] +"<br />"+df_sample['exon_id']
                y = df_sample['zScore']
                a = df_gene['start1'] +"<br />"+df_gene['exon_id']
                z = df_gene['zScore']
                
            fig = make_subplots(rows=1, cols=1)    
#            fig = px.line(x = x, y = y)
            fig.add_scatter(x=a, y = z, mode="markers",
                        marker=dict(color="darkgrey"), row=1, col=1, name='other samples', text=df_gene['sampleID'])

            gene_exon = df_sample['gene']
            print_ge = gene_exon
            if fragments=='exons' : 
                gene_exon = df_sample['exon_id']
                print_ge = "E"
            fig.add_scatter(x=x, y = y, 
                        marker=dict(color="blue"), row=1, col=1, name=current_sample_ID, mode="markers+text",
                        text = "<a href='gene_of_interest.ipynb?sample_id="+current_sample_ID+"&specie="+tissue+"&fragment=exons&gene_id="+gene_exon+"' target='_blank'>"+print_ge+"</a>", textposition = 'top center', textfont_size = 10)

#            fig = px.scatter(x = a, y = z)
            fig.update_xaxes(title_text="chr_start_position", showgrid=True)
            fig.update_yaxes(title_text="Z score", showgrid=False)
            fig.add_hline(y=0, line_dash="dash", line_color = 'grey')
            fig.update_layout(title= search_gene, title_x = 0.5, height=550, 
                      showlegend = True)

            fig.show()
            
    
gene_of_interest_plot(gene_ID)

## Ranking plot

This may take a while (up to a few minutes).
You can fill in a (unique) part of the geneID

In [39]:
# compare zScore of a fragment: chosen sample vs all samples
def ranking_plot(gene_ID):
    check_password()
    
    global df_gene
    
    if gene_ID == '':
        print('invalid gene_ID')
    
    else:
#        main_path = './outrider/'
#        file = 'umcu_rnaseq_' + tissue + '_res_outrider_' + fragments + '_counts.tsv.gz'
#        file = 'umcu_rnaseq_' + tissue + '_res_outrider_' + fragments + '_ref' + current_sample_ID +'.tsv.gz'
#        path = main_path + file

        df_gene = pd.read_csv(path)
        df_gene = stats.loc[df_gene['geneID'].str.contains(gene_ID, na = False, case = False)]
        
        df_gene.sort_values('normcounts', axis = 0, inplace=True, ascending =True)

        df_red = df_gene.loc[ df_gene['sampleID'] == current_sample_ID]

        clear_output()
        
        # plot
        fig = make_subplots(rows=1, cols=1)
        # all data
        fig.add_scatter(x=df_gene['sampleID'], y = df_gene['normcounts'], mode="markers",
                        marker=dict(color="darkgrey"),
                        row=1, col=1, text=df_gene['sampleID'])
        # sample data       
        fig.add_scatter(x = df_red['sampleID'], y = df_red['normcounts'],
                    marker=dict(color="Red"),
                    row=1, col=1, text=df_red['sampleID'])

        fig.update_traces(hoverinfo = 'text', selector = dict(type='scatter'), 
                           hoverlabel = dict(namelength = -1))
        fig.update_layout(title= gene_ID, title_x = 0.3, height=800, width = 1200, 
                          showlegend = False, template = "simple_white")
        fig.update_xaxes(title_text="sampleID", showgrid=False, automargin=True) #font smaller
        fig.update_yaxes(title_text="norm. counts", showgrid=False)

        fig.show()

        # display table
        display(HTML_with_style(df_gene[["sampleID","normcounts"]]))
    
ranking_plot(gene_ID)

interactive(children=(Text(value='', continuous_update=False, description='gene_ID:', placeholder='chr17_79649…

<function __main__.ranking_plot(gene_ID)>